# Get pVA and negLoss for all the desired models
Be aware that this is meant to get the pVA results for all the models that are inside in the cluster folder `/models/testing_things`.

In [1]:
import re
def extract_epoch_number(file_path):
    """
    Extracts the epoch number from a model weight file path.

    Args:
        file_path (str): The file path of the model weight file.

    Returns:
        int: The epoch number if found, otherwise None.
    """
    match = re.search(r'epoch(\d+)', file_path)
    if match:
        return int(match.group(1))
    return None

In [2]:
from utils.utils import AverageMeter
import time
import torch
from torch.autograd import Variable
from tqdm import tqdm
from utils.util import *
def get_silence_results(val_loader, model, device,epoch,args):
    pVA_aud_meter = AverageMeter()
    pVA_pad_meter = AverageMeter()
    pVA_meter = AverageMeter()
    sil_loss_meter = AverageMeter()


    model.eval()

    with torch.no_grad():
            
            for idx, batch in tqdm(enumerate(val_loader), total=len(val_loader)):
                image, spec, _, silence_vectors, nFrames = batch_unpacker(batch,args)

                spec = Variable(spec).to(device, non_blocking=True)
                image = Variable(image).to(device, non_blocking=True)
                silence_vectors = Variable(silence_vectors).to(device,non_blocking=True)  if args.punish_silence else None
                B = image.size(0)
            
                imgs_out, auds_out = model(image.float(), spec.float(), args, mode='val')

                imgs_out = imgs_out.to('cpu').detach()
                auds_out = auds_out.to('cpu').detach()
                T = auds_out.size(2)

                loss_sil, normal_mean = negAudio_loss(imgs_out,auds_out,silence_vectors)

                pVA_aud, pVA_pad, pVA, _, _ = measure_pVA(silence_vectors, imgs_out, auds_out, nFrames)
                
                pVA_aud_meter.update(pVA_aud.item())
                pVA_pad_meter.update(pVA_pad.item())
                pVA_meter.update(pVA.item())

                sil_loss_meter.update(loss_sil.item())
                print(f"Step: {idx}, pVA_aud: {pVA_aud.item()}, pVA_pad: {pVA_pad.item()}, pVA: {pVA.item()}, sil_loss: {loss_sil.item()}")

    if args.use_wandb:
        wandb.log({
            "pVA_aud": pVA_aud_meter.avg,
            "pVA_pad": pVA_pad_meter.avg,
            "pVA": pVA_meter.avg,
            "sil_loss" : sil_loss_meter.avg, 
            "epoch": epoch
        })

    return pVA_aud_meter.avg, pVA_pad_meter.avg, pVA_meter.avg, sil_loss_meter.avg

In [3]:
import os
from datasets import PlacesAudio
from torch.utils.data import  DataLoader
import re
# models
models_path = "testing_things"
results = {}

sys.argv = ['script_name', '--order_3_tensor', '--spec_DAVENet', '--padval_spec', '-80',
            '--placesAudio', '$DATA/PlacesAudio_400k_distro/metadata/',
            '--n_threads', '4',
            '--batch_size', '200',
            '--punish_silence',
            '--get_nFrames']


args = get_arguments()

# get the val_loader

val_dataset = PlacesAudio(args.placesAudio + 'val.json', args,mode='val')

print("Creating Data loaders with %d workers" % args.n_threads)
val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False,\
    num_workers=args.n_threads, drop_last=True, pin_memory=True)

# Walk through the directory and get all files
model_files = []
for root, dirs, files in os.walk(models_path):
    for model_path in files:
        epoch = extract_epoch_number(model_path)
        
        #Load model
        model, device = load_model(os.path.join(root,model_path),args)
        
        pVA_aud, pVA_pad, pVA, sil_loss = get_silence_results(val_loader, model, device, epoch, args)

        #Store in results
        results[model_path] = {
            "model_name": model_path,  # Add model_name as a column
            "pVA_aud": pVA_aud,
            "pVA_pad": pVA_pad,
            "pVA": pVA,
            "sil_loss": sil_loss,
            "epoch": epoch
        }
        print(f"Model: {model_path}, pVA_aud: {pVA_aud}, pVA_pad: {pVA_pad}, pVA: {pVA}, sil_loss: {sil_loss}")
        

    # Save results to a csv file
    import pandas as pd

    # Define the file path
    file_path = 'testing_things/silence_results.csv'
    # Create a DataFrame from the results dictionary
    df = pd.DataFrame.from_dict(results, orient='index')

    # Ensure model_name is the first column
    df.reset_index(drop=True, inplace=True)
    df = df[['model_name', 'pVA_aud', 'pVA_pad', 'pVA', 'sil_loss', 'epoch']]

    # Save the DataFrame to a CSV file
    df.to_csv(file_path, index=False)
    # Print the DataFrame
    print(df)




PlacesAudio split: VAL dataset size: 1000
Creating Data loaders with 4 workers


 20%|██        | 1/5 [00:20<01:21, 20.33s/it]

Step: 0, pVA_aud: 0.0010573742911219597, pVA_pad: 0.0010049270931631327, pVA: 0.0020623013842850924, sil_loss: 0.007841695100069046


 40%|████      | 2/5 [00:26<00:35, 11.90s/it]

Step: 1, pVA_aud: 0.0009713502367958426, pVA_pad: 0.0010007037781178951, pVA: 0.0019720541313290596, sil_loss: 0.006450706627219915


...

 80%|████████  | 4/5 [00:37<00:07,  7.63s/it]

Step: 3, pVA_aud: 0.0010720578720793128, pVA_pad: 0.0010135413613170385, pVA: 0.0020855991169810295, sil_loss: 0.007299158722162247


100%|██████████| 5/5 [00:43<00:00,  8.61s/it]

Step: 4, pVA_aud: 0.001038030837662518, pVA_pad: 0.0010427702218294144, pVA: 0.0020808009430766106, sil_loss: 0.006881620269268751
Model: SSL_TIE_PlacesAudio-lr1e-4-B128-MISA-SdT128-Sil-O1-epoch44.pth.tar, pVA_aud: 0.0010401288513094188, pVA_pad: 0.0010166259482502938, pVA: 0.0020567547995597123, sil_loss: 0.007161739747971296


                                          model_name  pVA_aud   pVA_pad  \
0  SSL_TIE_PlacesAudio-lr1e-4-B128-MISA-SdT128-Si...  0.00104  0.001017   

        pVA  sil_loss  epoch  
0  0.002057  0.007162     44  
